### **Summarization**

- Stuff: 전체 문서 한 번에 요약
- Map-Reduce: 분할 요약 후 일괄 병합
- Map-Refine: 분할 요약 후 점진적인 병합
- Chain of Density: N번 반복 실행하며, 누락된 entity를 보완하며 요약 개선
- Clustering-Map-Refine: 문서의 Chunk 를 N 개의 클러스터로 나누고, 각 클러스터에서 중심점에 가까운 문서에 대한 요약을 Refine 요약.

#### **대표적 요약 방법**

1. `Stuff`: 단순히 모든 문서를 단일 프롬프트로 "넣는" 방식입니다. 이는 가장 간단한 접근 방식

2. `Map-reduce`: 각 문서를 "map" 단계에서 개별적으로 요약한 다음, "reduce" 단계에서 요약본들을 최종 요약본으로 합치는 방식

3. `Refine`: 입력 문서를 순회하며 반복적으로 답변을 업데이트하여 응답을 구성합니다. 각 문서에 대해, 모든 비문서 입력, 현재 문서, 그리고 최신 중간 답변을 chain에 전달하여 새로운 답변 생성

### **1. Stuff**

- 전체 문서를 다 때려박아 한번에 요약
---

사용하기 쉬움 

Input Context가 문서량보다 작으면 안됨

In [8]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('data/news.txt')
docs = loader.load()

print(f'총 글자수 : {len(docs[0].page_content)}')

총 글자수 : 1708


In [11]:
# Hub에서 문서 요약 프롬프트 가져오기
from langchain import hub

prompt = hub.pull("teddynote/summary-stuff-documents-korean")
prompt.pretty_print()

Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. Translate the summary into KOREAN if it is written in ENGLISH.
5. DO NOT translate any technical terms.
6. DO NOT include any unnecessary information.

CONTEXT:
{context}

SUMMARY:"



In [25]:
from langchain_groq import ChatGroq
from langchain.chains.combine_documents import  create_stuff_documents_chain



llm = ChatGroq(
    model_name="llama3-70b-8192",
    temperature=0,
)


stuff_chain = create_stuff_documents_chain(llm, prompt)
answer = stuff_chain.stream({"context": docs})

for chunk in answer:
    print(chunk, end = '', flush=True)

Here is the summary of the article in Korean, following the requested format:

• 🎉 앨런AI연구소(AI2)가 완전한 오픈 소스 대형언어모델(LLM) '올모(OLMo)'를 출시했다.
• 📊 올모는 모델 코드, 모델 가중치, 훈련 코드, 훈련 데이터, 관련 툴킷 및 평가 툴킷을 제공하여 모델의 구축 과정을 심층적으로 분석할 수 있다.
• 🤖 올모 프레임워크는 4가지 변형 모델과 10억 매개변수의 '올모 1B' 모델을 제공하며, 아파치 2.0 라이선스에 따라 상업적 활용에 제한이 없다.
• 💡 카네기 멜론대학교는 빅테크의 오픈 소스 AI가 사실상 상업용 모델이라고 비난했으며, 실제 메타나 마이크로소프트, 애플 등이 공개한 오픈 소스는 연구에 한정되거나 상업 활용 시 사용자 수에 따라 별도 라이선스를 획득해야 한다.
• 💬 한나 하지시르지 AI2 올모 프로젝트 책임자는 "오늘날 많은 언어 모델은 제한된 투명성으로 공개된다"라며, "우리의 새로운 프레임워크를 통해 연구자들은 마침내 안전하고 신뢰할 수 있는 차세대 시스템을 구축하는 데 중요한 LLM 과학을 연구할 수 있게 될 것"이라고 말했다.
• 📈 올모는 상업용 제품과 동등한 성능을 보여주는 것으로 나타났으며, 메타의 '라마' 및 TII의 '팰컨'과 같은 모델과의 일부 자연어 처리 벤치마크 테스트에서 월등한 성능을 기록했다.
• 🚀 AI2는 "다양한 모델 크기, 양식, 데이터 세트 및 기능을 사용해 올모를 계속해서 향상할 것"이라고 말했다.
• 🌟 얀 르쿤 메타 수석 과학자는 "오픈 소스 기반 모델은 생성 AI에 대한 혁신과 개발을 촉진하는 데 매우 중요했다"며, "오픈 소스에서 비롯된 활발한 커뮤니티는 AI의 미래를 구축하는 가장 빠르고 효과적인 방법"이라고 말했다.

### **2. Map-Reduce**

Map-reduce 방식의 요약은 긴 문서를 효율적으로 요약하는 기법 

문서를 작은 chunk로 나누는 "map" 단계와, 각 chunk의 요약을 결합하는 "reduce" 단계로 구성

1. Map 단계에서는 각 chunk를 병렬로 요약하고(쪼개진 Chunk 마다의 요약을 생성)
2. reduce 단계에서는 이 요약들을 하나의 최종 요약으로 통합

이 접근법은 대규모 문서를 처리할 때 특히 유용하며, 언어 모델의 토큰 제한을 우회할 수 있게 해줌


<img src="./data/images/summarization_use_case_2.png" width="1000" alt="image">

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
docs = docs[3:8]  # 여기서 문서의 일부만 요약
print(f"총 페이지수: {len(docs)}")

총 페이지수: 5


### **2-1 Map**

각 Chunk 에 대한 요약 생성

In [33]:
from langchain import hub
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

llm = ChatGroq(
    temperature=0,
    model_name="gemma2-9b-it",
)

# map prompt 다운로드
map_prompt = hub.pull("teddynote/map-prompt")

# 프롬프트 출력
# map_prompt.pretty_print()

# Map Chain 
map_chain = prompt | llm | StrOutputParser()

# 문서에 대한 주요내용 추출
doc_summaries = map_chain.batch(docs)

# 요약된 문서의 수 출력
len(doc_summaries)

5

### **2-2 Reduce**

Reduce 단계에서는 map 단계에서 진행한 핵심 내용들을 하나의 최종 요약으로 통합

In [34]:
# reduce prompt 다운로드
reduce_prompt = hub.pull("teddynote/reduce-prompt")

# 프롬프트 출력
# reduce_prompt.pretty_print()

# reduce chain 생성
reduce_chain = reduce_prompt | llm | StrOutputParser()

answer = reduce_chain.stream(
    {"doc_summaries": "\n".join(doc_summaries), "language": "Korean"}
)

for chunk in answer:
    print(chunk, end="", flush=True)

## 미국 AI 관련 정책 및 논의 요약

* **AI 안전 및 보안:**
    * 강력한 AI 시스템 개발 기업은 안전 테스트 결과와 시스템 정보를 미국 정부에 공유해야 함.
    * AI 시스템 안전성 및 신뢰성 확인을 위한 표준 및 AI 생성 콘텐츠 표시 표준 마련 추진.
    * 개인정보보호 강화 조치.
* **형평성과 시민권:**
    * 법률, 주택, 보건 분야에서 AI로 인한 차별 및 편견 방지 조치 확대.
    * 형사사법 시스템에서 AI 사용 모범사례 개발 및 주택 임대 시 AI 알고리즘 차별 방지 지침 제공.
* **소비자 보호 및 근로자 지원:**
    * 의료 분야에서 책임 있는 AI 사용 촉진 및 맞춤형 개인교습 등 학교 내 AI 교육 도구 개발.
    * AI로 인한 근로자 피해 완화 및 이점 극대화 원칙 및 모범사례 마련.
* **혁신과 경쟁 촉진:**
    * 국가 AI 연구자원(NAIRR)을 통해 미국 전역의 AI 연구 촉진 및 중소기업과 개발자에 기술 및 인프라 지원.
    * 국가 차원의 AI 연구 인프라 확충 및 AI 관련 분야 전문가 유치 지원.
* **국제 협력:**
    * G7 국가들은 AI 국제 행동강령에 합의, AI 수명주기 전반에 걸쳐 위험 평가와 완화, 투명성과 책임성 보장, 정보공유 및 이해관계자 협력, 보안 통제, 콘텐츠 인증 및 출처 확인 등의 조치 요구.
    * 28개국 참여 AI 안전성 정상회의에서 AI 안전 보장을 위한 국가, 국제기구, 기업, 시민사회, 학계 등 모든 이해관계자의 협력 강조.
* **저작권 논쟁:**
    * 미국 법원, 예술가들의 AI 기업 저작권 소송 기각.
    * AI 모델 학습에 원작자 동의 없이 작품을 사용했다는 주장은 인정되지 않음.
* **FTC의 우려:**
    * FTC는 생성 AI가 소비자와 창작자에게 피해를 줄 수 있다고 우려하며, 개인정보 침해, 차별과 편견의 자동화, 사기 범죄 등의 위험에 주목.
    * 빅테크 기업의 AI 시장

In [9]:
# 하나로 합침
from langchain_core.runnables import chain
from langchain_teddynote.callbacks import StreamingCallback
from langchain_groq import ChatGroq
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

@chain
def map_reduce_chain(docs):
    map_llm = ChatGroq(
        temperature=0,
        model_name="gemma2-9b-it",
    )

    # map prompt 다운로드
    map_prompt = hub.pull("teddynote/map-prompt")

    # map chain 생성
    map_chain = map_prompt | map_llm | StrOutputParser()

    # 첫 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    doc_summaries = map_chain.batch(docs)

    # reduce prompt 다운로드
    reduce_prompt = hub.pull("teddynote/reduce-prompt")
    reduce_llm = ChatGroq(
        model_name="llama-3.3-70b-versatile",
        temperature=0,
        callbacks=[StreamingCallback()],
        streaming=True,
    )

    reduce_chain = reduce_prompt | reduce_llm | StrOutputParser()

    return reduce_chain.invoke(
        {"doc_summaries": "\n".join(doc_summaries), "language": "Korean"}
    )
        
map_reduce_chain.invoke(docs)

요약:
* 미국 정부는 바이든 대통령이 안전하고 신뢰할 수 있는 인공지능(AI) 개발과 사용을 보장하기 위한 행정 명령을 발동했습니다.
* 행정 명령은 AI의 안전과 보안 표준, 개인 정보 보호, 공정성과 민권, 소비자와 노동자 보호, 혁신과 경쟁 촉진, 국제 협력 강화 등 7개 분야를 강조합니다.
* G7 국가들은 고급 AI 시스템 개발과 배포의 위험을 해결하기 위한 국제 행동 강령을 채택했습니다.
* 블레츨리 선언은 모든 이해관계자의 협력과 첨단 AI 시스템 개발 기업의 안전 평가 및 적절한 조치의 중요성을 강조합니다.
* 미국 법원은 AI 아트 회사에 대한 저작권 침해 소송을 기각했지만, Stability AI에 대한 별도의 소송은 진행 중입니다.
* 연방거래위원회(FTC)는 생성형 AI로 인한 소비자와 창작자의 잠재적 피해에 대한 우려를 표명하고, 규제와 정책의 필요성을 강조했습니다.

'요약:\n* 미국 정부는 바이든 대통령이 안전하고 신뢰할 수 있는 인공지능(AI) 개발과 사용을 보장하기 위한 행정 명령을 발동했습니다.\n* 행정 명령은 AI의 안전과 보안 표준, 개인 정보 보호, 공정성과 민권, 소비자와 노동자 보호, 혁신과 경쟁 촉진, 국제 협력 강화 등 7개 분야를 강조합니다.\n* G7 국가들은 고급 AI 시스템 개발과 배포의 위험을 해결하기 위한 국제 행동 강령을 채택했습니다.\n* 블레츨리 선언은 모든 이해관계자의 협력과 첨단 AI 시스템 개발 기업의 안전 평가 및 적절한 조치의 중요성을 강조합니다.\n* 미국 법원은 AI 아트 회사에 대한 저작권 침해 소송을 기각했지만, Stability AI에 대한 별도의 소송은 진행 중입니다.\n* 연방거래위원회(FTC)는 생성형 AI로 인한 소비자와 창작자의 잠재적 피해에 대한 우려를 표명하고, 규제와 정책의 필요성을 강조했습니다.'

### **3. Map-Refine**

1. Map 단계: 문서를 여러 개의 작은 chunk로 나누고, 각 chunk에 대해 개별적으로 요약을 생성(여기까지 Map-Reduce와 같음)

2. Refine 단계: 생성된 요약들을 순차적으로 처리하며 최종 요약을 점진적으로 개선 각 단계에서 이전 요약과 새로운 chunk의 정보를 결합 하여 요약 갱신
   
3. 반복 과정: 모든 chunk가 처리될 때까지 refine 단계를 반복

4. 최종 요약: 마지막 chunk까지 처리한 후 얻은 요약이 최종 결과

- map-refine 방식은 문서 순서를 유지하며 점진적으로 요약을 개선할 수 있어 문서 맥락이 중요한 경우 유용
- 순차 처리로 인해 병렬화가 어려워 대규모 문서 처리 시 시간이 더 소요됩니다.

<img src="./data/images/summarization_use_case_3.png" width="1000" alt="image">


In [11]:
from langchain_core.runnables import chain

@chain
def map_refine_chain(docs):

    # map chain 생성
    map_summary = hub.pull("teddynote/map-summary-prompt")

    map_chain = (
        map_summary
        | ChatGroq(
            model_name="gemma2-9b-it",
            temperature=0,
        )
        | StrOutputParser()
    )

    input_doc = [{"documents": doc.page_content, "language": "Korean"} for doc in docs]

    # 첫 번째 프롬프트, ChatGroq, 문자열 출력 파서를 연결하여 체인을 생성합니다.
    doc_summaries = map_chain.batch(input_doc)

    refine_prompt = hub.pull("teddynote/refine-prompt")

    refine_llm = ChatGroq(
        model_name="gemma2-9b-it",
        temperature=0,
        callbacks=[StreamingCallback()],
        streaming=True,
    )

    refine_chain = refine_prompt | refine_llm | StrOutputParser()
    
    # 이전 문서 요약을 현재 문서 요약에 적용(첫 문서)
    previous_summary = doc_summaries[0]

    for current_summary in doc_summaries[1:]:

        previous_summary = refine_chain.invoke(
            {
                "previous_summary": previous_summary,
                "current_summary": current_summary,
                "language": "Korean",
            }
        )
        print("\n\n-----------------\n\n")

    return previous_summary

map_refine_chain.invoke(docs)

바이든 대통령은 안전하고 신뢰할 수 있는 인공지능(AI) 개발과 사용을 위한 행정명령을 발표했습니다. 이 행정명령은 AI 안전 및 보안 기준 마련, 개인정보보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제협력을 주요 목표로 합니다. 특히 강력한 AI 시스템 개발 기업은 안전 테스트 결과와 시스템 정보를 정부에 공유해야 하며, AI 시스템의 안전성과 신뢰성을 확인하기 위한 표준 마련을 추진합니다. 또한, 법률, 주택, 보건 분야에서 AI의 무책임한 사용으로 인한 차별과 편견을 방지하고, AI로 인한 근로자 피해를 완화하는 조치를 강조합니다. 

이는 G7 국가들이 2023년 10월 히로시마 AI 프로세스를 통해 첨단 AI 시스템 개발 기업을 대상으로 채택한 AI 국제 행동강령과도 부합합니다. G7 국가들은 AI 수명주기 전반에 걸친 위험 평가와 완화, 투명성과 책임성, 정보공유 및 협력, 보안 통제, 콘텐츠 인증 등을 강조하는 이 행동강령을 통해 AI 기술의 윤리적 개발과 안전한 활용을 위한 기준을 마련하고 있습니다. 




-----------------


바이든 대통령은 안전하고 신뢰할 수 있는 인공지능(AI) 개발과 사용을 위한 행정명령을 발표했습니다. 이 행정명령은 AI 안전 및 보안 기준 마련, 개인정보보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제협력을 주요 목표로 합니다. 특히 강력한 AI 시스템 개발 기업은 안전 테스트 결과와 시스템 정보를 정부에 공유해야 하며, AI 시스템의 안전성과 신뢰성을 확인하기 위한 표준 마련을 추진합니다. 또한, 법률, 주택, 보건 분야에서 AI의 무책임한 사용으로 인한 차별과 편견을 방지하고, AI로 인한 근로자 피해를 완화하는 조치를 강조합니다. 

이는 G7 국가들이 2023년 10월 히로시마 AI 프로세스를 통해 첨단 AI 시스템 개발 기업을 대상으로 채택한 AI 국제 행동강령과도 부합합니다. G7 국가들은 AI 수명주기 전반에 걸친 위험 

"바이든 대통령은 안전하고 신뢰할 수 있는 인공지능(AI) 개발과 사용을 위한 행정명령을 발표했습니다. 이 행정명령은 AI 안전 및 보안 기준 마련, 개인정보보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제협력을 주요 목표로 합니다. 특히 강력한 AI 시스템 개발 기업은 안전 테스트 결과와 시스템 정보를 정부에 공유해야 하며, AI 시스템의 안전성과 신뢰성을 확인하기 위한 표준 마련을 추진합니다. 또한, 법률, 주택, 보건 분야에서 AI의 무책임한 사용으로 인한 차별과 편견을 방지하고, AI로 인한 근로자 피해를 완화하는 조치를 강조합니다. \n\n이는 G7 국가들이 2023년 10월 히로시마 AI 프로세스를 통해 첨단 AI 시스템 개발 기업을 대상으로 채택한 AI 국제 행동강령과도 부합합니다. G7 국가들은 AI 수명주기 전반에 걸친 위험 평가와 완화, 투명성과 책임성, 정보공유 및 협력, 보안 통제, 콘텐츠 인증 등을 강조하는 이 행동강령을 통해 AI 기술의 윤리적 개발과 안전한 활용을 위한 기준을 마련하고 있습니다. \n\n최근 영국에서 개최된 AI 안전성 정상회의에서는 28개국이 AI 안전 보장을 위한 협력 방안을 담은 '블레츨리 선언'을 발표했습니다. 선언은 AI 시스템 개발 기업의 안전 평가 책임 강조와 첨단 AI 개발 기업의 투명성 향상, 안전 테스트 도구 개발 등의 협력 필요성을 제시했습니다. 영국은 첨단 AI 모델 안전성 시험 계획 수립과 테스트 수행을 주도할 AI 안전 연구소 출범을 발표했으며, 참석국들은 튜링상 수상자 요슈아 벤지오 교수가 주도하는 '과학의 현황' 보고서 작성을 통해 첨단 AI 위험과 가능성을 과학적으로 평가하기로 합의했습니다. 한국은 영국과 온라인 AI 미니 정상회의를 공동 개최하고, 프랑스와는 대면 정상회의를 개최하기로 합의했습니다. \n\n미국 법원은 예술가들이 AI 기업에 제기한 저작권 침해 소송을 일부 기각했습니다. 고소된 작품의 저작권 등록이 부족하고, AI 생성 이미지와 특정 작

## Chain of Density

- 논문: https://arxiv.org/pdf/2309.04269

"Chain of Density" (CoD) 프롬프트는 GPT-4를 사용한 요약 생성을 개선하기 위해 개발된 기법입니다. 

이 방법은 초기에 개체가 적은 요약을 생성한 후, 길이를 늘리지 않으면서 누락된 중요 개체들을 반복적으로 통합하는 과정을 거칩니다. 연구 결과, CoD로 생성된 요약은 일반 프롬프트보다 더 추상적이고 정보 융합이 뛰어나며, 인간이 작성한 요약과 비슷한 밀도를 가진 것으로 나타났습니다.

1. 점진적 개선: CoD는 초기에 개체가 적은 간단한 요약을 생성한 후, 단계적으로 중요한 개체들을 추가하며 요약을 개선합니다. 이 과정에서 요약의 길이는 유지되면서 정보 밀도가 증가하여 읽기 쉬우면서도 정보량이 풍부한 요약이 만들어집니다.

2. 정보 밀도와 가독성의 균형: CoD 방식은 요약의 정보 밀도를 조절하여 정보성과 가독성 사이의 최적 균형점을 찾습니다. 연구 결과에 따르면, 사람들은 일반적인 GPT-4 요약보다 더 밀도 있지만 사람이 작성한 요약만큼 밀도가 높지 않은 CoD 요약을 선호하는 것으로 나타났습니다.

3. 추상화와 정보 융합 개선: CoD로 생성된 요약은 더 추상적이고 정보 융합이 뛰어나며, 원문의 앞부분에 치우치는 경향(lead bias)이 덜합니다. 이는 요약의 전반적인 품질과 가독성을 향상시키는 데 기여합니다.

[Chain of Density Prompt](https://smith.langchain.com/prompts/chain-of-density-prompt/4582aae0?organizationId=8c9eeb3c-2665-5405-bc50-0767fdf4ca8f)

**입력 파라미터 설명**

- `content_category`: 콘텐츠 정류(예: 기사, 동영상 녹취록, 블로그 게시물, 연구 논문). 기본값: Article

- `content`: 요약할 콘텐츠

- `entity_range`: 콘텐츠에서 선택하여 요약에 추가할 엔티티의 수의 범위. 기본값은 `1-3`

- `max_words`: 1번 요약시, 요약에 포함할 최대 단어. 기본값은 **80** 입니다.

- `iterations`: 엔티티 고밀도화 라운드 수. 총 요약은 **반복 횟수+1** 입니다. 80단어의 경우 3회 반복이 이상적입니다. 요약이 더 길면 4~5회, 그리고 `entity_range` 를 예를 들어 1~4로 변경하는 것도 도움이 될 수 있습니다. 기본값: 3.

In [ ]:
# Chain of Density 프롬프트 다운로드
cod_prompt = hub.pull("teddynote/chain-of-density-prompt")

cod_prompt.pretty_print()

In [ ]:
import textwrap
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import SimpleJsonOutputParser

# {content}를 제외한 모든 입력에 대한 기본값 지정
cod_chain_inputs = {
    "content": lambda d: d.get("content"),
    "content_category": lambda d: d.get("content_category", "Article"),
    "entity_range": lambda d: d.get("entity_range", "1-3"),
    "max_words": lambda d: int(d.get("max_words", 80)),
    "iterations": lambda d: int(d.get("iterations", 5)),
}

# Chain of Density 프롬프트 다운로드
cod_prompt = hub.pull("teddynote/chain-of-density-prompt")

# Chain of Density 체인 생성
cod_chain = (
    cod_chain_inputs
    | cod_prompt
    | ChatOpenAI(temperature=0, model="gpt-4o-mini")
    | SimpleJsonOutputParser()
)

# 두 번째 체인 생성, 최종 요약만 추출 (스트리밍 불가능, 최종 결과가 필요함)
cod_final_summary_chain = cod_chain | (
    lambda output: output[-1].get(
        "denser_summary", '오류: 마지막 딕셔너리에 "denser_summary" 키가 없습니다'
    )
)

In [ ]:
# 결과를 저장할 빈 리스트 초기화
results: list[dict[str, str]] = []

# cod_chain을 스트리밍 모드로 실행하고 부분적인 JSON 결과를 처리
for partial_json in cod_chain.stream(
    {"content": content, "content_category": "Article"}
):
    # 각 반복마다 results를 업데이트
    results = partial_json

    # 현재 결과를 같은 줄에 출력 (캐리지 리턴을 사용하여 이전 출력을 덮어씀)
    print(results, end="\r", flush=True)

# 총 요약 수 계산
total_summaries = len(results)
print("\n")

# 각 요약을 순회하며 처리
i = 1
for cod in results:
    # 누락된 엔티티들을 추출하고 포맷팅
    added_entities = ", ".join(
        [
            ent.strip()
            for ent in cod.get(
                "missing_entities", 'ERR: "missing_entiies" key not found'
            ).split(";")
        ]
    )
    # 더 밀도 있는 요약 추출
    summary = cod.get("denser_summary", 'ERR: missing key "denser_summary"')

    # 요약 정보 출력 (번호, 총 개수, 추가된 엔티티)
    print(
        f"### CoD Summary {i}/{total_summaries}, 추가된 엔티티(entity): {added_entities}"
        + "\n"
    )
    # 요약 내용을 80자 너비로 줄바꿈하여 출력
    print(textwrap.fill(summary, width=80) + "\n")
    i += 1

print("\n============== [최종 요약] =================\n")
print(summary)